In [1]:
import pickle
import numpy as np
import pandas as pd
from keras import models
from keras.layers import Dense, Concatenate
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
import time
import os
import sys
import CNN_functions as CNNFuncs

In [2]:
test_size = 0.2
random_state = 1
model_mode = "depmap"
GOIs_mode = "top100"
extract_genes_no = 5000
extract_neg_selec = True

activation_func = 'relu' 
activation_func2 = 'linear'
kernel_initializer = 'he_uniform'
dense_layer_dim = 250
l2_reg = 0.001
batch_size = 500
num_epoch = 50

save_model = False

In [3]:
datasets, essential_genes = CNNFuncs.preprocess_data(test_size=test_size, 
                                                     random_state=random_state, 
                                                     mode=model_mode, 
                                                     extract_top_genes=extract_genes_no, 
                                                     extract_neg_selec=extract_neg_selec)

X_train = datasets[0]
Y_train = datasets[2]

X_test = datasets[1]
Y_test = datasets[3]

top100_essential_genes = essential_genes["top100_essential_genes"]
top_common_essential_genes = essential_genes["common_essential_genes"]

DepMap dataset row/column check:  {'all_match': True, 'column_differences': [[], []], 'row_differences': [[], []]}
tRCC dataset row/column check:  {'all_match': True, 'column_differences': [[], []], 'row_differences': [[], []]}
Check columns across DepMap and tRCC dataframes:  {'all_match': True, 'column_differences': [[], [], [], []]}


In [4]:
X_train, Y_train, X_test, Y_test = CNNFuncs.create_model_datasets(X_train=X_train, Y_train=Y_train, X_test=X_test, Y_test=Y_test, GOIs= top_common_essential_genes if GOIs_mode == "top_common" else top100_essential_genes)

In [5]:
X_train

,A3GALT2,AAAS,AAMP,AARD,AARS1,AARS2,AARSD1,AASDHPPT,AATF,AATK,...,ZRSR2,ZSCAN18,ZSCAN20,ZSCAN4,ZSWIM8,ZW10,ZWILCH,ZWINT,ZYG11B,ZZZ3
0,0.0,5.876517,6.671576,0.831877,7.799476,3.509696,5.627899,3.760221,5.311067,1.384050,...,2.313246,1.000000,1.182692,0.014355,5.200457,3.190615,4.601697,7.030888,2.035624,3.760221
1,0.0,4.754888,7.268285,0.000000,6.767258,2.042644,5.431957,5.680043,6.451211,0.097611,...,4.052242,0.000000,1.201634,1.929791,4.685380,3.572890,2.829850,2.675816,2.797013,4.293518
2,0.0,5.687621,6.723012,0.000000,4.964861,2.922198,6.351734,5.134632,6.503349,3.148934,...,3.316146,0.056584,1.395063,0.000000,5.004501,4.058316,4.631686,6.964745,2.859970,3.797013
3,0.0,5.286881,6.729417,0.389567,7.050502,3.629939,6.125982,4.305241,6.087251,3.003602,...,3.416840,3.631104,1.765535,0.000000,5.925287,4.281698,4.573496,6.503349,2.364572,4.422233
4,0.0,4.930264,6.971314,0.000000,7.554666,2.711495,5.273889,4.952334,7.076281,0.485427,...,4.121844,4.416164,1.056584,0.275007,4.468583,4.266037,3.932628,5.606442,3.891419,5.173127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81495,0.0,5.541019,6.202809,0.042644,7.016697,4.498889,6.904725,6.550901,4.945327,3.061776,...,4.314697,5.175924,1.321928,0.189034,5.107269,4.238023,5.083639,7.053220,3.177918,4.280214
81496,0.0,4.639811,6.565445,0.070389,6.794416,3.112700,5.041331,3.770829,5.770565,0.443607,...,3.645010,4.550285,1.207893,0.176323,5.545660,3.806324,4.694880,6.857981,2.283922,3.524816
81497,0.0,4.699330,6.728329,0.070389,7.454916,2.153805,6.057234,5.366322,6.457627,0.422233,...,3.158660,0.485427,1.835924,0.042644,5.521679,4.380591,4.161081,6.724241,3.331992,4.780835
81498,0.0,5.725741,6.747522,0.000000,6.974415,3.578939,6.072963,4.858478,6.024364,0.678072,...,2.263034,4.397803,0.400538,0.014355,5.284292,3.945795,4.574102,6.516015,1.627607,3.468583


In [6]:
model = models.Sequential()
model.add(Dense(500, input_dim=X_train.shape[1], activation=activation_func, kernel_initializer=kernel_initializer, kernel_regularizer=l2(l2_reg)))
model.add(Dense(200, activation=activation_func, kernel_initializer=kernel_initializer, kernel_regularizer=l2(l2_reg)))
model.add(Dense(50, activation=activation_func, kernel_initializer=kernel_initializer, kernel_regularizer=l2(l2_reg)))
model.add(Dense(dense_layer_dim, activation=activation_func, kernel_initializer=kernel_initializer, kernel_regularizer=l2(l2_reg)))
model.add(Dense(dense_layer_dim, activation=activation_func, kernel_initializer=kernel_initializer, kernel_regularizer=l2(l2_reg)))
model.add(Dense(1, activation=activation_func2, kernel_initializer=kernel_initializer, kernel_regularizer=l2(l2_reg)))

model.compile(loss='mse', optimizer='adam') 

t = time.time()

early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')
history = model.fit(X_train, Y_train, epochs=num_epoch, validation_split=1/9, batch_size=batch_size, shuffle=True, callbacks=[early_stopping_callback])

cost_testing = model.evaluate(X_test, Y_test, verbose=0, batch_size=batch_size)

loss = history.history['loss'][-1]
val_loss = history.history['val_loss'][-1]

print("\n\nModel training completed in %.1f mins.\nloss:%.4f valloss:%.4f testloss:%.4f" % ((time.time() - t)/60, loss, val_loss, cost_testing))

# Save the model
# model.save("/results/models/model_demo.h5")
# print("\n\nModel saved in /results/models/model_demo.h5\n\n")



Epoch 1/10
145/145 [==============================] - 3s 17ms/step - loss: 74.3461 - val_loss: 1.9015
Epoch 2/10
145/145 [==============================] - 2s 16ms/step - loss: 1.8892 - val_loss: 1.8012
Epoch 3/10
145/145 [==============================] - 2s 16ms/step - loss: 3.7397 - val_loss: 1.7198
Epoch 4/10
145/145 [==============================] - 2s 16ms/step - loss: 1.5863 - val_loss: 1.6714
Epoch 5/10
145/145 [==============================] - 2s 16ms/step - loss: 1.5380 - val_loss: 1.6243
Epoch 6/10
145/145 [==============================] - 2s 16ms/step - loss: 1.5047 - val_loss: 1.5902
Epoch 7/10
145/145 [==============================] - 2s 16ms/step - loss: 1.5253 - val_loss: 1.5741
Epoch 8/10
145/145 [==============================] - 2s 16ms/step - loss: 1.4734 - val_loss: 1.8024
Epoch 9/10
145/145 [==============================] - 2s 16ms/step - loss: 1.4256 - val_loss: 1.5054
Epoch 10/10
145/145 [==============================] - 2s 17ms/step - loss: 1.3846 - val_l

In [7]:
predictions = model.predict(X_test)

predictions_df = pd.DataFrame(predictions, columns=['Predicted_Score'])

print(predictions_df)

638/638 [==============================] - 1s 2ms/step
       Predicted_Score
0            -0.620085
1            -0.733947
2            -0.669031
3            -0.728986
4            -0.607613
...                ...
20395        -0.700834
20396        -0.609333
20397        -0.647066
20398        -0.706081
20399        -0.615499

[20400 rows x 1 columns]


In [8]:
Y_test

0        0.061965
1       -0.369769
2        0.263827
3       -0.076322
4        0.327040
           ...   
20395   -0.003899
20396   -0.459931
20397   -0.468172
20398   -0.192836
20399   -0.558814
Length: 20400, dtype: float64